In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

calc_emergence =False
debris_only=False

min_dhdt_perc = 75

debug=False
extra_layers=True

csv_ending = '_mb_bins.csv'
# outdir_csv = debris_prms.mb_binned_fp
outdir_csv = ('/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/' + 
              '../output/mb_bins_all/csv/')
outdir_csv = outdir_csv + debris_prms.roi + '/'
if not os.path.exists(outdir_csv):
    os.makedirs(outdir_csv)
    
# dhdt_vel_fns_fp = debris_prms.dhdt_vel_fns_fp
dhdt_vel_fns_fp = ('/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/' + 
                   '../output/dhdt_vel_fns_all/')
if not os.path.exists(dhdt_vel_fns_fp):
    os.makedirs(dhdt_vel_fns_fp)


In [3]:
if debris_only:
    # Debris cover extent shapefile with statistics
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])

    # Subset by percent debris-covered or debris-covered area
    dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                            (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                            & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
    dc_shp_subset.reset_index(inplace=True, drop=True)
    dc_shp_subset['CenLon_360'] = dc_shp_subset['CenLon']
    dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'] = (
        360 + dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'])

    rgiid_list = [x.split('-')[1] for x in dc_shp_subset['RGIId'].values]
    main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgiid_list)

else:
    # All glaciers
    main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgi_regionsO1=debris_prms.roi_rgidict[debris_prms.roi],
                                                              rgi_regionsO2='all', rgi_glac_number='all')
    
    # Debris cover extent shapefile with statistics
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])
    dc_shp.reset_index(inplace=True, drop=True)

# dc_shp
main_glac_rgi_subset

All glaciers within region(s) 5 are included in this model run.
This study is focusing on 20261 glaciers in region [5]


,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,0,RGI60-05.00001,-53.2909,66.0765,5,1,0.168,689,995,868,29.0,25,406,0,0,9,20000814,1,05.00001,5.00001
1,1,RGI60-05.00002,-52.0705,65.3617,5,1,0.118,602,704,643,14.0,49,283,0,0,9,20000823,2,05.00002,5.00002
2,2,RGI60-05.00003,-52.0017,65.5791,5,1,0.059,646,697,683,10.1,28,365,0,0,9,20000823,3,05.00003,5.00003
3,3,RGI60-05.00004,-51.7992,65.1590,5,1,0.188,635,775,735,11.8,30,392,0,0,9,20000823,4,05.00004,5.00004
4,4,RGI60-05.00005,-51.7228,65.1786,5,1,0.086,711,769,747,13.1,1,218,0,0,9,20000823,5,05.00005,5.00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20256,20256,RGI60-05.20275,-29.7831,70.2707,5,1,1.008,1226,1560,1502,12.6,21,1093,1,0,9,20010921,20275,05.20275,5.20275
20257,20257,RGI60-05.20277,-29.7908,70.2829,5,1,0.071,1238,1282,1256,7.7,70,175,0,0,9,20010921,20277,05.20277,5.20277
20258,20258,RGI60-05.20278,-28.9625,69.9847,5,1,1.459,955,2066,1430,42.8,359,1697,1,0,9,20010712,20278,05.20278,5.20278


In [4]:
dhdt_vel_fns_fn = debris_prms.dhdt_vel_fns_fn.replace('XXXX',debris_prms.roi)
if os.path.exists(dhdt_vel_fns_fp + dhdt_vel_fns_fn):
    dhdt_vel_fns_df = pd.read_csv(dhdt_vel_fns_fp + dhdt_vel_fns_fn)
else:
    dhdt_vel_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_subset.shape[0], 3)),
                                   columns=['RGIId', 'dhdt_fullfn', 'vel_fullfn'])
    dhdt_vel_fns_df['RGIId'] = main_glac_rgi_subset['RGIId']

In [5]:
# np.where(main_glac_rgi_subset.rgino_str == '11.03005')

In [6]:
# skip existing
rgiid_existing = []
for i in os.listdir(outdir_csv):
    if i.endswith('.csv'):
        rgiid_existing.append(i.split('_')[0])
print('existing:', len(rgiid_existing))

existing: 0


In [ ]:
# ===== PROCESS EACH GLACIER =====
dc_rgiids = dc_shp.RGIId.tolist()
for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
# for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values[83926:]):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[120]]): # Miage
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[2307]]): # Ngozumpa

    glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
    rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
    region = glac_str.split('.')[0]

    if int(region) < 10:
        glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
    else:
        glac_str_noleadzero = glac_str
        
    if glac_str_noleadzero not in rgiid_existing:

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        
        if os.path.exists(thick_dir + thick_fn):
        
            gf = create_glacfeat(thick_dir, thick_fn)

            if rgiid in dc_rgiids:
                # Debris shape layer processing
                dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                                  str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
                if os.path.exists(dc_shp_proj_fn) == False:
                    dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                    dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                    dc_shp_single = dc_shp_single.reset_index()
                    dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                    dc_shp_proj.to_file(dc_shp_proj_fn)
                dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
                dc_shp_lyr = dc_shp_ds.GetLayer()
            else:
                dc_shp_lyr = None

            # ==== CHECK IF TIF HAS DHDT DATA OVER THE GLACIER =====
            mb_fullfns = []
            find_mb = True
            dhdt_fn_wglacier = None
            for mb_fp in debris_prms.mb_fp_list_roi[debris_prms.roi]:
                if find_mb:
                    for i in os.listdir(mb_fp):
                        if i.endswith('.tif'):
                            mb_fullfns.append(mb_fp + i)
                    tif_count = 0
                    while find_mb and tif_count < len(mb_fullfns):
                        dhdt_fn = mb_fullfns[tif_count]
                        if debug:
                            print(tif_count, dhdt_fn.split('/')[-1])

                        # Add the filenames
                        fn_dict = OrderedDict()
                        # DEM
                        z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
                        z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
                        fn_dict['z1'] = z1_fp + z1_fn
                        # Ice thickness
                        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
                        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
                        fn_dict['ice_thick'] = thick_dir + thick_fn
                        # dh/dt
                        fn_dict['dhdt'] = dhdt_fn
                        # ===== PROCESS THE DATA =====
                        #Expand extent to include buffered region around glacier polygon
                        warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
                        #Warp everything to common res/extent/proj
                        z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
                        z1_res = np.min([z1_gt[1], -z1_gt[5]])
                        ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                           t_srs=gf.aea_srs, verbose=False, r='cubic')
                        ds_dict = dict(zip(fn_dict.keys(), ds_list))
                        gf.ds_dict = ds_dict

                        if 'z1' in ds_dict:
                            #This is False over glacier polygon surface, True elsewhere - can be applied directly
                            glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                            glac_geom_mask_copy = glac_geom_mask.copy()
                            gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                            if rgiid in dc_rgiids:
                                # Debris cover
                                dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                                gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                            if 'dhdt' in ds_dict:
                                gf.dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                gf.dhdt.mask = np.ma.mask_or(
                                    glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.dhdt.data, 
                                                                                     np.isnan(gf.dhdt.data))))
                                
                                if rgiid in dc_rgiids:
                                    gf.dc_dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=gf.dc_mask)
                                    gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)
                                # Count dhdt pixels
                                dhdt_pixels = len(gf.dhdt.nonzero()[0])                              
                                
                                if dhdt_pixels / gf.z1.count() * 100 > min_dhdt_perc:
                                    dhdt_fn_wglacier = dhdt_fn
                                    find_mb = False
                                    if debug:
                                        print('\n# z1 pixels:', gf.z1.count())
                                        print('# dhdt_pixels:', dhdt_pixels)
                                        var_full2plot = gf.dhdt.copy()
                                        clim = malib.calcperc(var_full2plot, (2,98))
                                        plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', 
                                                   close_fig=False)
                        # Loop over layers        
                        tif_count += 1

            # ==== CHECK IF VELOCITY DATA OVER THE GLACIER =====
            vel_fullfns = []
            find_vel = True
            vx_fn_wglacier = None
            if find_vel and dhdt_fn_wglacier is not None:
                vx_fns = debris_prms.vx_dir_dict_list[debris_prms.roi]
                tif_count = 0
                while find_vel and tif_count < len(vx_fns):
                    vx_fn = vx_fns[tif_count]

                    if debug:
                        print(tif_count, vx_fn.split('/')[-1])

                    # Add the filenames
                    # Velocity
                    fn_dict['vx'] = vx_fn
                    fn_dict['vy'] = vx_fn.replace('_vx', '_vy')
                    # ===== PROCESS THE DATA =====
                    ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                       t_srs=gf.aea_srs, verbose=False, r='cubic')
                    ds_dict = dict(zip(fn_dict.keys(), ds_list))
                    gf.ds_dict = ds_dict
                    if 'vx' in ds_dict and 'vy' in ds_dict:
                        #Load surface velocity maps
                        gf.vx = np.ma.array(iolib.ds_getma(ds_dict['vx']), mask=glac_geom_mask)
                        gf.vy = np.ma.array(iolib.ds_getma(ds_dict['vy']), mask=glac_geom_mask)
                        gf.vm = np.ma.sqrt(gf.vx**2 + gf.vy**2)
                        # Count velocity pixels
                        vel_pixels = len(gf.vm.nonzero()[0])
                        if debug:
                                print('\n# z1 pixels:', gf.z1.count())
                                print('# vel_pixels:', vel_pixels)
                                var_full2plot = gf.vm.copy()
                                clim = malib.calcperc(var_full2plot, (2,98))
                                plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vm (m/yr)', 
                                           close_fig=False)
                        if vel_pixels / gf.z1.count() * 100 > min_dhdt_perc:
                            vx_fn_wglacier = vx_fn
                            find_vel = False

                    # Loop over layers        
                    tif_count += 1


            # ===== Add layers =====
            if dhdt_fn_wglacier is not None and vx_fn_wglacier is not None:
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=True, dhdt_fn=dhdt_fn_wglacier, gf_add_vel=True, 
                              vx_fn=vx_fn_wglacier, gf_add_ts=False, gf_add_slope_aspect=True, gf_add_ts_info=False, 
                              calc_emergence=calc_emergence, debug_emergence=False)
                # Save dhdt and vel filenames
                dhdt_vel_fns_df.loc[glac_idx,:] = [rgiid, dhdt_fn_wglacier, vx_fn_wglacier]

                # ===== PLOTS =====
#                 plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    # Elevation change
                    var_full2plot = gf.dhdt.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', close_fig=False)
                    # Velocity
                    var_full2plot = gf.vm.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vel (m/yr)', close_fig=False)
                    # Emergence velocity
                    if gf.emvel is not None:
                        var_full2plot = gf.emvel.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' emvel'], 'inferno', 'emvel (m/yr)', close_fig=False)
                    # Surface temperature
                    if gf.ts is not None:
                        var_full2plot = gf.ts.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)

                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:7] + csv_ending)
                else:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)
            
            # Elevation bins for glaciers with only dhdt data
            elif dhdt_fn_wglacier is not None and vx_fn_wglacier is None:
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=True, dhdt_fn=dhdt_fn_wglacier, gf_add_vel=False, 
                              gf_add_ts=False, gf_add_slope_aspect=True, gf_add_ts_info=False, 
                              calc_emergence=calc_emergence, debug_emergence=False)
                # Save dhdt and vel filenames
                dhdt_vel_fns_df.loc[glac_idx,:] = [rgiid, dhdt_fn_wglacier, 0]

                # ===== PLOTS =====
#                 plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    # Elevation change
                    var_full2plot = gf.dhdt.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', close_fig=False)
                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:7] + csv_ending)
                else:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)
            
            # Elevation bins for glaciers without any data
            else:
                gf.dhdt, gf.dc_dhdt = None, None
                gf.vx, gf.vy, gf.vm = None, None, None
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=False, gf_add_vel=False, gf_add_ts=False, 
                              gf_add_slope_aspect=True, gf_add_ts_info=False, 
                              calc_emergence=calc_emergence, debug_emergence=False,
                             verbose=False)
                # ===== PLOTS =====
#                 plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    
                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                outdir_csv_nodhdt = outdir_csv + 'no_dhdt/'
                csv_ending_nodhdt = '_bins.csv'
                if not os.path.exists(outdir_csv_nodhdt):
                    os.makedirs(outdir_csv_nodhdt)
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv_nodhdt, gf.feat_fn[0:7] + csv_ending_nodhdt)
                else:
                    outbins_fullfn = os.path.join(outdir_csv_nodhdt, gf.feat_fn[0:8] + csv_ending_nodhdt)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)


# Save updated filenames
dhdt_vel_fns_df.to_csv(dhdt_vel_fns_fp + dhdt_vel_fns_fn, index=False)

0 0 RGI60-05.00001
1 1 RGI60-05.00002
2 2 RGI60-05.00003
3 3 RGI60-05.00004
4 4 RGI60-05.00005
5 5 RGI60-05.00006
6 6 RGI60-05.00007
7 7 RGI60-05.00008
8 8 RGI60-05.00009
9 9 RGI60-05.00010
10 10 RGI60-05.00011
11 11 RGI60-05.00012
12 12 RGI60-05.00013
13 13 RGI60-05.00014
14 14 RGI60-05.00015
15 15 RGI60-05.00016
16 16 RGI60-05.00017
17 17 RGI60-05.00018
18 18 RGI60-05.00019
19 19 RGI60-05.00020
20 20 RGI60-05.00021
21 21 RGI60-05.00022
22 22 RGI60-05.00023
23 23 RGI60-05.00024
24 24 RGI60-05.00025
25 25 RGI60-05.00026
26 26 RGI60-05.00027
27 27 RGI60-05.00028
28 28 RGI60-05.00029
29 29 RGI60-05.00030
30 30 RGI60-05.00031
31 31 RGI60-05.00032
32 32 RGI60-05.00033
33 33 RGI60-05.00034
34 34 RGI60-05.00035
35 35 RGI60-05.00036
36 36 RGI60-05.00037
37 37 RGI60-05.00038
38 38 RGI60-05.00039
39 39 RGI60-05.00040
40 40 RGI60-05.00041
41 41 RGI60-05.00042
42 42 RGI60-05.00043
43 43 RGI60-05.00044
44 44 RGI60-05.00045
45 45 RGI60-05.00046
46 46 RGI60-05.00047
47 47 RGI60-05.00048
48 48 RGI60-

366 366 RGI60-05.00367
367 367 RGI60-05.00368
368 368 RGI60-05.00369
369 369 RGI60-05.00370
370 370 RGI60-05.00371
371 371 RGI60-05.00372
372 372 RGI60-05.00373
373 373 RGI60-05.00374
374 374 RGI60-05.00375
375 375 RGI60-05.00376
376 376 RGI60-05.00377
377 377 RGI60-05.00378
378 378 RGI60-05.00379
379 379 RGI60-05.00380
380 380 RGI60-05.00381
381 381 RGI60-05.00382
382 382 RGI60-05.00383
383 383 RGI60-05.00384
384 384 RGI60-05.00385
385 385 RGI60-05.00386
386 386 RGI60-05.00387
387 387 RGI60-05.00388
388 388 RGI60-05.00389
389 389 RGI60-05.00390
390 390 RGI60-05.00391
391 391 RGI60-05.00392
392 392 RGI60-05.00393
393 393 RGI60-05.00394
394 394 RGI60-05.00395
395 395 RGI60-05.00396
396 396 RGI60-05.00397
397 397 RGI60-05.00398
398 398 RGI60-05.00399
399 399 RGI60-05.00400
400 400 RGI60-05.00401
401 401 RGI60-05.00402
402 402 RGI60-05.00403
403 403 RGI60-05.00404
404 404 RGI60-05.00405
405 405 RGI60-05.00406
406 406 RGI60-05.00407
407 407 RGI60-05.00408
408 408 RGI60-05.00409
409 409 RGI

723 723 RGI60-05.00724
724 724 RGI60-05.00725
725 725 RGI60-05.00726
726 726 RGI60-05.00727
727 727 RGI60-05.00728
728 728 RGI60-05.00729
729 729 RGI60-05.00730
730 730 RGI60-05.00731
731 731 RGI60-05.00732
732 732 RGI60-05.00733
733 733 RGI60-05.00734
734 734 RGI60-05.00735
735 735 RGI60-05.00736
736 736 RGI60-05.00737
737 737 RGI60-05.00738
738 738 RGI60-05.00739
739 739 RGI60-05.00740
740 740 RGI60-05.00741
741 741 RGI60-05.00742
742 742 RGI60-05.00743
743 743 RGI60-05.00744
744 744 RGI60-05.00745
745 745 RGI60-05.00746
746 746 RGI60-05.00747
747 747 RGI60-05.00748
748 748 RGI60-05.00749
749 749 RGI60-05.00750
750 750 RGI60-05.00751
751 751 RGI60-05.00752
752 752 RGI60-05.00753
753 753 RGI60-05.00754
754 754 RGI60-05.00755
755 755 RGI60-05.00756
756 756 RGI60-05.00757
757 757 RGI60-05.00758
758 758 RGI60-05.00759
759 759 RGI60-05.00760
760 760 RGI60-05.00761
761 761 RGI60-05.00762
762 762 RGI60-05.00763
763 763 RGI60-05.00764
764 764 RGI60-05.00765
765 765 RGI60-05.00766
766 766 RGI

1073 1073 RGI60-05.01074
1074 1074 RGI60-05.01075
1075 1075 RGI60-05.01076
1076 1076 RGI60-05.01077
1077 1077 RGI60-05.01078
1078 1078 RGI60-05.01079
1079 1079 RGI60-05.01080
1080 1080 RGI60-05.01081
1081 1081 RGI60-05.01082
1082 1082 RGI60-05.01083
1083 1083 RGI60-05.01084
1084 1084 RGI60-05.01085
1085 1085 RGI60-05.01086
1086 1086 RGI60-05.01087
1087 1087 RGI60-05.01088
1088 1088 RGI60-05.01089
1089 1089 RGI60-05.01090
1090 1090 RGI60-05.01091
1091 1091 RGI60-05.01092
1092 1092 RGI60-05.01093
1093 1093 RGI60-05.01094
1094 1094 RGI60-05.01095
1095 1095 RGI60-05.01096
1096 1096 RGI60-05.01097
1097 1097 RGI60-05.01098
1098 1098 RGI60-05.01099
1099 1099 RGI60-05.01100
1100 1100 RGI60-05.01101
1101 1101 RGI60-05.01102
1102 1102 RGI60-05.01103
1103 1103 RGI60-05.01104
1104 1104 RGI60-05.01105
1105 1105 RGI60-05.01106
1106 1106 RGI60-05.01107
1107 1107 RGI60-05.01108
1108 1108 RGI60-05.01109
1109 1109 RGI60-05.01110
1110 1110 RGI60-05.01111
1111 1111 RGI60-05.01112
1112 1112 RGI60-05.01113


1401 1401 RGI60-05.01402
1402 1402 RGI60-05.01403
1403 1403 RGI60-05.01404
1404 1404 RGI60-05.01405
1405 1405 RGI60-05.01406
1406 1406 RGI60-05.01407
1407 1407 RGI60-05.01408
1408 1408 RGI60-05.01409
1409 1409 RGI60-05.01410
1410 1410 RGI60-05.01411
1411 1411 RGI60-05.01412
1412 1412 RGI60-05.01413
1413 1413 RGI60-05.01414
1414 1414 RGI60-05.01415
1415 1415 RGI60-05.01416
1416 1416 RGI60-05.01417
1417 1417 RGI60-05.01418
1418 1418 RGI60-05.01419
1419 1419 RGI60-05.01420
1420 1420 RGI60-05.01421
1421 1421 RGI60-05.01422
1422 1422 RGI60-05.01423
1423 1423 RGI60-05.01424
1424 1424 RGI60-05.01425
1425 1425 RGI60-05.01426
1426 1426 RGI60-05.01427
1427 1427 RGI60-05.01428
1428 1428 RGI60-05.01429
1429 1429 RGI60-05.01430
1430 1430 RGI60-05.01431
1431 1431 RGI60-05.01432
1432 1432 RGI60-05.01433
1433 1433 RGI60-05.01434
1434 1434 RGI60-05.01435
1435 1435 RGI60-05.01436
1436 1436 RGI60-05.01437
1437 1437 RGI60-05.01438
1438 1438 RGI60-05.01439
1439 1439 RGI60-05.01440
1440 1440 RGI60-05.01441


...

5011 5011 RGI60-05.05012
5012 5012 RGI60-05.05013
5013 5013 RGI60-05.05014
5014 5014 RGI60-05.05015
5015 5015 RGI60-05.05016
5016 5016 RGI60-05.05017
5017 5017 RGI60-05.05018
5018 5018 RGI60-05.05019
5019 5019 RGI60-05.05020
5020 5020 RGI60-05.05021
5021 5021 RGI60-05.05022
5022 5022 RGI60-05.05023
5023 5023 RGI60-05.05024
5024 5024 RGI60-05.05025
5025 5025 RGI60-05.05026
5026 5026 RGI60-05.05027
5027 5027 RGI60-05.05028
5028 5028 RGI60-05.05029
5029 5029 RGI60-05.05030
5030 5030 RGI60-05.05031
5031 5031 RGI60-05.05032
5032 5032 RGI60-05.05033
5033 5033 RGI60-05.05034
5034 5034 RGI60-05.05035
5035 5035 RGI60-05.05036
5036 5036 RGI60-05.05037
5037 5037 RGI60-05.05038
5038 5038 RGI60-05.05039
5039 5039 RGI60-05.05040
5040 5040 RGI60-05.05041
5041 5041 RGI60-05.05042
5042 5042 RGI60-05.05043
5043 5043 RGI60-05.05044
5044 5044 RGI60-05.05045
5045 5045 RGI60-05.05046
5046 5046 RGI60-05.05047
5047 5047 RGI60-05.05048
5048 5048 RGI60-05.05049
5049 5049 RGI60-05.05050
5050 5050 RGI60-05.05051


5339 5339 RGI60-05.05340
5340 5340 RGI60-05.05341
5341 5341 RGI60-05.05342
5342 5342 RGI60-05.05343
5343 5343 RGI60-05.05344
5344 5344 RGI60-05.05345
5345 5345 RGI60-05.05346
5346 5346 RGI60-05.05347
5347 5347 RGI60-05.05348
5348 5348 RGI60-05.05349
5349 5349 RGI60-05.05350
5350 5350 RGI60-05.05351
5351 5351 RGI60-05.05352
5352 5352 RGI60-05.05353
5353 5353 RGI60-05.05354
5354 5354 RGI60-05.05355
5355 5355 RGI60-05.05356
5356 5356 RGI60-05.05357
5357 5357 RGI60-05.05358
5358 5358 RGI60-05.05359
5359 5359 RGI60-05.05360
5360 5360 RGI60-05.05361
5361 5361 RGI60-05.05362
5362 5362 RGI60-05.05363
5363 5363 RGI60-05.05364
5364 5364 RGI60-05.05365
5365 5365 RGI60-05.05366
5366 5366 RGI60-05.05367
5367 5367 RGI60-05.05368
5368 5368 RGI60-05.05369
5369 5369 RGI60-05.05370
5370 5370 RGI60-05.05371
5371 5371 RGI60-05.05372
5372 5372 RGI60-05.05373
5373 5373 RGI60-05.05374
5374 5374 RGI60-05.05375
5375 5375 RGI60-05.05376
5376 5376 RGI60-05.05377
5377 5377 RGI60-05.05378
5378 5378 RGI60-05.05379


5667 5667 RGI60-05.05668
5668 5668 RGI60-05.05669
5669 5669 RGI60-05.05670
5670 5670 RGI60-05.05671
5671 5671 RGI60-05.05672
5672 5672 RGI60-05.05673
5673 5673 RGI60-05.05674
5674 5674 RGI60-05.05675
5675 5675 RGI60-05.05676
5676 5676 RGI60-05.05677
5677 5677 RGI60-05.05678
5678 5678 RGI60-05.05679
5679 5679 RGI60-05.05680
5680 5680 RGI60-05.05681
5681 5681 RGI60-05.05682
5682 5682 RGI60-05.05683
5683 5683 RGI60-05.05684
5684 5684 RGI60-05.05685
5685 5685 RGI60-05.05686
5686 5686 RGI60-05.05687
5687 5687 RGI60-05.05688
5688 5688 RGI60-05.05689
5689 5689 RGI60-05.05690
5690 5690 RGI60-05.05691
5691 5691 RGI60-05.05692
5692 5692 RGI60-05.05693
5693 5693 RGI60-05.05694
5694 5694 RGI60-05.05695
5695 5695 RGI60-05.05696
5696 5696 RGI60-05.05697
5697 5697 RGI60-05.05698
5698 5698 RGI60-05.05699
5699 5699 RGI60-05.05700
5700 5700 RGI60-05.05701
5701 5701 RGI60-05.05702
5702 5702 RGI60-05.05703
5703 5703 RGI60-05.05704
5704 5704 RGI60-05.05705
5705 5705 RGI60-05.05706
5706 5706 RGI60-05.05707


5995 5995 RGI60-05.05996
5996 5996 RGI60-05.05997
5997 5997 RGI60-05.05998
5998 5998 RGI60-05.05999
5999 5999 RGI60-05.06000
6000 6000 RGI60-05.06001
6001 6001 RGI60-05.06002
6002 6002 RGI60-05.06003
6003 6003 RGI60-05.06004
6004 6004 RGI60-05.06005
6005 6005 RGI60-05.06006
6006 6006 RGI60-05.06007
6007 6007 RGI60-05.06008
6008 6008 RGI60-05.06009
6009 6009 RGI60-05.06010
6010 6010 RGI60-05.06011
6011 6011 RGI60-05.06012
6012 6012 RGI60-05.06013
6013 6013 RGI60-05.06014
6014 6014 RGI60-05.06015
6015 6015 RGI60-05.06016
6016 6016 RGI60-05.06017
6017 6017 RGI60-05.06018
6018 6018 RGI60-05.06019
6019 6019 RGI60-05.06020
6020 6020 RGI60-05.06021
6021 6021 RGI60-05.06022
6022 6022 RGI60-05.06023
6023 6023 RGI60-05.06024
6024 6024 RGI60-05.06025
6025 6025 RGI60-05.06026
6026 6026 RGI60-05.06027
6027 6027 RGI60-05.06028
6028 6028 RGI60-05.06029
6029 6029 RGI60-05.06030
6030 6030 RGI60-05.06031
6031 6031 RGI60-05.06032
6032 6032 RGI60-05.06033
6033 6033 RGI60-05.06034
6034 6034 RGI60-05.06035


6323 6323 RGI60-05.06324
6324 6324 RGI60-05.06325
6325 6325 RGI60-05.06326
6326 6326 RGI60-05.06327
6327 6327 RGI60-05.06328
6328 6328 RGI60-05.06329
6329 6329 RGI60-05.06330
6330 6330 RGI60-05.06331
6331 6331 RGI60-05.06332
6332 6332 RGI60-05.06333
6333 6333 RGI60-05.06334
6334 6334 RGI60-05.06335
6335 6335 RGI60-05.06336
6336 6336 RGI60-05.06337
6337 6337 RGI60-05.06338
6338 6338 RGI60-05.06339
6339 6339 RGI60-05.06340
6340 6340 RGI60-05.06341
6341 6341 RGI60-05.06342
6342 6342 RGI60-05.06343
6343 6343 RGI60-05.06344
6344 6344 RGI60-05.06345
6345 6345 RGI60-05.06346
6346 6346 RGI60-05.06347
6347 6347 RGI60-05.06348
6348 6348 RGI60-05.06349
6349 6349 RGI60-05.06350
6350 6350 RGI60-05.06351
6351 6351 RGI60-05.06352
6352 6352 RGI60-05.06353
6353 6353 RGI60-05.06354
6354 6354 RGI60-05.06355
6355 6355 RGI60-05.06356
6356 6356 RGI60-05.06357
6357 6357 RGI60-05.06358
6358 6358 RGI60-05.06359
6359 6359 RGI60-05.06360
6360 6360 RGI60-05.06361
6361 6361 RGI60-05.06362
6362 6362 RGI60-05.06363


In [ ]:
print('\n\nDONE!\n\n')

In [ ]:
# skip existing
rgiid_existing = []
for i in os.listdir(outdir_csv):
    if i.endswith('.csv'):
        rgiid_existing.append(i.split('_')[0])
main_glac_rgi_processed = debris_prms.selectglaciersrgitable(rgiid_existing)
print('Processed ' + str(len(rgiid_existing)) + ' of ' + str(len(main_glac_rgi_subset)) + ' glaciers')
area_total = main_glac_rgi_subset.Area.sum()
area_processed = main_glac_rgi_processed.Area.sum()
print('  Area: ' + str(np.round(area_processed,1)) + ' km2 (' + str(np.round(area_processed / area_total * 100,1)) + 
      '%) of ' +  str(np.round(area_total,1)) + ' km2')

In [29]:
# # Isolate HMA regions
# # outdir_csv_raw = outdir_csv.replace('HMA/', '')
# if debris_prms.roi == 'HMA':
#     for region in ['13','14','15']:
#         outdir_csv = outdir_csv_raw + region + '/'
#         rgiid_existing = []
#         for i in os.listdir(outdir_csv):
#             if i.endswith('.csv'):
#                 rgiid_existing.append(i.split('_')[0])
#         main_glac_rgi_processed = debris_prms.selectglaciersrgitable(rgiid_existing)
#         main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgi_regionsO1=[int(region)],
#                                                                   rgi_regionsO2='all', rgi_glac_number='all')
#         area_total = main_glac_rgi_subset.Area.sum()
#         area_processed = main_glac_rgi_processed.Area.sum()
#         print('Processed ' + str(len(rgiid_existing)) + ' of ' + str(len(main_glac_rgi_subset)) + ' glaciers')
#         print('  Area: ' + str(np.round(area_processed,1)) + ' km2 (' + str(np.round(area_processed / area_total * 100,1)) + 
#               '%) of ' +  str(np.round(area_total,1)) + ' km2\n')

In [10]:
# ===== SHEAN ESTIMATE OF FLUX DIVERGENCE QUICKLY ======
#                 if gf.H is not None:
#                     #Compute flux
#                     gf.Q = gf.H * debris_prms.v_col_f * np.array([gf.vx, gf.vy])
#                     #Note: np.gradient returns derivatives relative to axis number, so (y, x) in this case
#                     #Want x-derivative of x component
#                     gf.divQ = np.gradient(gf.Q[0])[1] + np.gradient(gf.Q[1])[0]
# #                     gf.divQ = gf.H*(np.gradient(v_col_f*gf.vx)[1] + np.gradient(v_col_f*gf.vy)[0]) \
# #                             + v_col_f*gf.vx*(np.gradient(gf.H)[1]) + v_col_f*gf.vy*(np.gradient(gf.H)[0])
#                     #Should smooth divQ, better handling of data gaps

In [ ]:
# ===== OLD CHECK DEM FOR ERRORS AND REPLACE SCRIPT (no longer needed with OGGM processing) =====
#         #Create buffer around glacier polygon
#         glac_geom_buff = gf.glac_geom.Buffer(debris_prms.buff_dist)
#         #This is False over glacier polygon surface, True elsewhere - can be applied directly
#         glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])
        
#             # ds masks
#             ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#             dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
#             dems_mask = dem1.mask
#             if verbose:
#                 print('list of datasets:', len(ds_list_masked), fn_dict.values())

#             #Combine to identify ~1 km buffer around glacier polygon over static rock
#             static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#             static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)
        
        
#             # Check if DEM has huge errors or not - replace if necessary
#             if input.roi in ['01']:

#                 gf.z1_check = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
#                 if gf.z1_check.min() < 0:

#                     # Add backup DEM for regions with known poor quality (ex. Alaska)
#                     print('switching DEMs')
#                     fn_dict['z1_backup'] = input.z1_backup_dict[input.roi]
#                     # Warp everything to common res/extent/proj (a second time)
#                     ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, \
#                             extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
#                             r='cubic')
#                     ds_dict = dict(zip(fn_dict.keys(), ds_list))

#                     if verbose:
#                         print(ds_list)
#                         print(fn_dict.keys())

#                     # ds masks
#                     ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#                     dem1 = np.ma.masked_less_equal(ds_list_masked[-1], 0)
#                     dems_mask = dem1.mask
#                     if verbose:
#                         print('list of datasets:', len(ds_list_masked), fn_dict.values())

#                     #Combine to identify ~1 km buffer around glacier polygon over static rock
#                     static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#                     static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

#                     #This is False over glacier polygon surface, True elsewhere - can be applied directly
#                     glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1_backup'])
#                     gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=glac_geom_mask)
#                     #gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)

#                     # Debris cover
#                     dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
#                     gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=dc_mask)

In [7]:
rgi_shp = gpd.read_file('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/qgis_himat/rgi60_HMA.shp')
hma_fp = '/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/output/mb_bins_all/csv/HMA/'
rgiids = []
for i in os.listdir(hma_fp):
    if i.endswith('.csv'):
        rgiids.append('RGI60-' + i.split('_')[0])
rgiids = sorted(rgiids)

rgiids_all = list(rgi_shp.RGIId.values)
rgiids_missing = list(set(rgiids_all) - set(rgiids))

rgiids_missing_idx = []
for nglac, x in enumerate(rgi_shp.RGIId.values):
    if x in rgiids_missing:
        rgiids_missing_idx.append(nglac)
        
rgi_shp_missing = rgi_shp.loc[rgiids_missing_idx,:]
# rgi_shp_missing

output_fn = '/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/output/hma_missing.shp'
rgi_shp_missing.to_file(output_fn)